# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Read Test Images

In [2]:
img_color = cv2.imread('test_imgs/jenga_tower_color.png')
img_depth = cv2.imread('test_imgs/jenga_tower_depth.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


## Extract Blocks' Masks By Colors

In [3]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [4]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [5]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 70, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,70,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 50, 50)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 50, 50)
upper_violet = (130+20, 255, 255)

In [6]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    if color == 'violet':
        cv2.imshow('src_bin', src_bin)
        cv2.imshow(f'{color}_filtered', each_color_filtered)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 600:
            continue
        
        if color == 'violet':
            cv2.imshow('blk clr', block_color)
            # cv2.imwrite('./test_imgs/red_block1_rgb.png', block_color)
            cv2.imshow('blk msk', block_mask)
            # cv2.imwrite('./test_imgs/red_block1_mask.png', block_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [ ]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [ ]:
cv2.imshow('tower mask', tower_mask)
# cv2.imwrite('./test_imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
# cv2.imwrite('./test_imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
6 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [ ]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[971.179, 0, 1025.07],[0, 970.984, 778.291],[0, 0, 1]]
intrinsic.intrinsic_matrix = [[971.179, 0, 1025.07],[0, 970.984, 778.291],[0, 0, 1]]

In [ ]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [ ]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [ ]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [ ]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 115893 points.
[[1.34147933e-04 7.52568333e-06 6.82352984e-04]
 [1.34850535e-04 7.52568333e-06 6.82352984e-04]
 [1.35553138e-04 7.52568333e-06 6.82352984e-04]
 ...
 [8.46677371e-05 2.42506470e-04 4.47058817e-04]
 [8.51280630e-05 2.42506470e-04 4.47058817e-04]
 [8.55883889e-05 2.42506470e-04 4.47058817e-04]]


In [ ]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(512, 0.0001)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='green':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [ ]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [ ]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

o3d.visualization.draw_geometries([pcd_combined])

PointCloud with 113320 points.


In [ ]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

pcd_target = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_target)
o3d.visualization.draw_geometries([pcd_target])

TriangleMesh with 12 points and 4 triangles.
PointCloud with 113320 points.


In [ ]:
def prepare_icp(source, target):
    source_tmp = copy.deepcopy(source)
    target_tmp = copy.deepcopy(target)
    
    # make the point cloud into right position
    trans_init = np.asarray([[0, 0, -1, 0],
                             [0, -1, 0, 0],
                             [-1, 0, 0, 0],
                             [0, 0, 0, 1]])
    
    source_tmp.transform(trans_init)
    
    # resize the target pointcloud to make two pointclouds into same scale
    resize = (np.array(target_tmp.points)[:,1].max() - np.array(target_tmp.points)[:,1].min()) / (np.array(source_tmp.points)[:,1].max() - np.array(source_tmp.points)[:,1].min())
    
    # move the source pcd to do icp
    move = np.array(target_tmp.get_center() - source_tmp.get_center()*resize)
    
    # a = o3d.cpu.pybind.utility.Vector3dVector(np.array(target_tmp.points))
    b = o3d.cpu.pybind.utility.Vector3dVector(np.array(source_tmp.points)*resize + move)
    
    # target_tmp.points = a
    source_tmp.points = b
    
    o3d.visualization.draw_geometries([source_tmp, target_tmp])
    
    return source_tmp, target_tmp, resize

In [ ]:
source, target, resize = prepare_icp(pcd_combined, pcd_target)

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
threshold = 10
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

In [ ]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=5.325891e-01, inlier_rmse=4.927920e+00, and correspondence_set size of 60353
Access transformation to get result.


In [ ]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=20000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=9.921550e-01, inlier_rmse=1.310553e+00, and correspondence_set size of 112431
Access transformation to get result.
Transformation is:
[[ 0.80823603 -0.02603788 -0.58828271 18.94908214]
 [ 0.05796727  0.99768773  0.03548219 -2.10932773]
 [ 0.58599856 -0.06277913  0.80787652  1.69153206]
 [ 0.          0.          0.          1.        ]]


In [ ]:
draw_registration_result(source, target, reg_p2p.transformation)

In [ ]:
source_temp = copy.deepcopy(source)

source_temp.transform(reg_p2p.transformation)


PointCloud with 113320 points.

In [ ]:
(np.array(source_temp.points))[:,1].max()

172.3565868079812

## Block ICP

In [ ]:
for idx, pcd_y in enumerate(blocks_pcd_yellow):
    exec(f"pcd_yellow_{idx} = pcd_y")

for idx, pcd_y in enumerate(blocks_pcd_pink):
    exec(f"pcd_pink_{idx} = pcd_y")

# pcd_yellow_1 = blocks_pcd_yellow[1]
# print(pcd_yellow_1)

In [ ]:
mesh_block = o3d.io.read_triangle_mesh("mesh/single_block_side_z.stl")
print(mesh_block)

mesh_block.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

pcd_target = mesh_block.sample_points_uniformly(number_of_points=len(pcd_yellow_1.points))
print(pcd_target)
o3d.visualization.draw_geometries([pcd_target])

TriangleMesh with 96 points and 32 triangles.
PointCloud with 6780 points.


In [ ]:
source, target, resize = prepare_icp(pcd_yellow_1, pcd_target)

In [ ]:
threshold = 105
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

In [ ]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=4.056047e-01, inlier_rmse=2.549297e+00, and correspondence_set size of 2750
Access transformation to get result.


In [ ]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=20000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=5.383481e-01, inlier_rmse=2.713453e+00, and correspondence_set size of 3650
Access transformation to get result.
Transformation is:
[[ 0.94735773 -0.31919888 -0.0250083   6.18490747]
 [ 0.31235907  0.938557   -0.14677384  0.61825315]
 [ 0.07032176  0.13123576  0.98885389 -0.92020799]
 [ 0.          0.          0.          1.        ]]


In [ ]:
np.array(target.points)[:,1].max()

14.997922939281592

In [ ]:
np.array(pcd_yellow_1.points)[:,1].max() * resize

56.48950919174722

In [ ]:
for m in blocks_mask_by_color[2]:
    cv2.imshow('m', m)
    # cv2.imshow(f'{color}_filtered', each_color_filtered)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
y_list1 = []
for i in range(6):
    exec(f"print(np.array(pcd_yellow_{i}.points)[:,1].max() * resize)")
    exec(f"y_list1.append(pcd_yellow_{i})")

31.829490698143406
56.50084041162012
81.62043314712827
106.25294630666482
131.59515906063885
157.04081889734178


In [ ]:
trans_init2 = np.asarray([[0, 0, -1, 0],
                          [0, -1, 0, 0],
                          [-1, 0, 0, 0],
                          [0, 0, 0, 1]])

In [ ]:
y_list2 = []
for i in range(6):
    exec(f"tmp = copy.deepcopy(pcd_yellow_{i})")
    tmp.transform(trans_init2)
    print(np.array(tmp.points)[:,1].max() * resize)
    y_list2.append(tmp)

-18.088182544636314
-41.5037044133849
-65.69857790705865
-89.59571129954578
-114.74975587352819
-139.46368142116125


In [ ]:
o3d.visualization.draw_geometries(y_list2)

In [ ]:
np.array(pcd_yellow_0.points)[:,0].std() * resize

17.098270015234572

In [ ]:
np.array(pcd_pink_0.points)[:,0].std() * resize

3.1581143023328466

In [ ]:
np.array(pcd_yellow_0.points)[:,1].max() * resize

31.829490698143406